# Mapping Floods

## Introduction



### IALS  - 16.11.2024

In [1]:
#!pip install hydrafloods

In [7]:
import ee

In [25]:
ee.Authenticate(auth_mode='gcloud')

True

In [26]:
ee.Initialize(project='eo-salcedobett')

In [27]:
import geemap

In [ ]:
#geemap.get_ee_token()

In [29]:
#geemap.get_ee_token()

In [30]:
import datetime
import hydrafloods as hf
import geemap.foliumap as geemap

In [31]:
Map = geemap.Map(center=[10.5, -75], zoom=8)

In [32]:
colombia = (
    ee.FeatureCollection("FAO/GAUL/2015/level0")
    .filter("ADM0_NAME == 'Colombia'")
    .filter(ee.Filter.eq('ADM0_NAME', 'Colombia'))
       )

In [33]:
colestilo = {
  'fillColor': 'b5ffb4',
  'color': '00909F',
  'width': 1.0,
}

In [34]:
region = hf.country_bbox("Colombia")

In [35]:
startTime = datetime.datetime(2024,11,1)
endTime = datetime.datetime(2024,11,13)

In [36]:
ls8 = hf.Landsat8(region, startTime, endTime)

## Import SAR and optical imagery

In [37]:
s1 = hf.Sentinel1(region, startTime, endTime)

In [38]:
s1.n_images

110

In [39]:
ls8.n_images

65

## Extract DEM and Height Above Nearest Drainage (HAND)

In [40]:
merit = ee.Image("MERIT/Hydro/v1_0_1")

In [41]:
dem = merit.select("elv").unmask(0)

In [42]:
hand = merit.select("hnd").unmask(0)

## Pseudo-terrain flattening algorithm

In [43]:
s1_flat = s1.apply_func(hf.slope_correction, elevation = dem, 
                        buffer = 50, model="surface")

## Apply a speckle filter

In [44]:
s1_filtered = s1.apply_func(hf.gamma_map)

## Mask any area that is greater than 20 m HAND

In [45]:
s1_lowelv = s1_filtered.apply_func(lambda x: x.updateMask(hand.lt(20)))

## Apply a water thresholding algorithm

In [46]:
water = s1_lowelv.apply_func(hf.edge_otsu, initial_threshold= -16,
                             band ="VV", edge_buffer=300, scale=90)

## Reduce the collection using the mode reducer

In [47]:
water_img = water.collection.reduce("mode")

## Map S-1 Imagery to see effects of terrain flattening and HAND masking

In [48]:
Map = geemap.Map(center= (5.69, -76.65), zoom=10)

In [49]:
Map.addLayer(s1_flat.collection.median(),
             {"bands": "VV", "min":-25, "max":0},
             'Sentinel 1 (flat)')

In [50]:
Map.addLayer(s1_lowelv.collection.median(),
             {"bands": "VV", "min":-25, "max":0},
             'Sentinel 1 (HAND masked)')

In [51]:
Map.addLayerControl()
Map

## Compare with permanent water

In [52]:
permanent_water = (
    ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
    .filterDate('2000-01-01', endTime)
    .limit(5, "system:time_start", False)
    .map(lambda x: x.select("waterClass").eq(3))
    .sum()
    .unmask(0)
    .gt(0)
               ).updateMask(water_img.gte(0))

## Find where the surface water is flooded

In [53]:
floods = water_img.add(permanent_water).eq(1)

## Map SAR floods

In [54]:
Map = geemap.Map(center= (5.69, -76.65), zoom=10)
Map.addLayer(s1.collection.median(),
             {"bands": "VV", "min":-25, "max":0},
             'Sentinel 1')
Map.addLayer(permanent_water.selfMask(),
             {"min":0, "max":1,"palette":["ffffff",  "0000ff"]},
             'Permanent Water')
Map.addLayer(floods.selfMask(),
             {"min":0, "max":1,"palette":["ffffff", "ff0000"]},
             'Floods')
Map.addLayerControl()
Map